# 1. Gerekli Kütüphanelerin Yüklenmesi


In [1]:
# Gerekli kütüphaneleri yükleme
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


# 2. Veri Setinin Hazırlanması
## 2.1 Sınıfların Seçimi ve Dengelenmesi

In [2]:
# Veri seti klasörlerinin yolları
source_dir = "/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"  
target_dir = "/kaggle/working/FilteredImages"  # Filtrelenmiş veri seti kaydedileceği yol

# Kullanılacak sınıflar
selected_classes = ["collie", "dolphin", "elephant", "fox", "moose", "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]
images_per_class = 650

# Veriyi filtreleme ve yeni klasöre kopyalama
os.makedirs(target_dir, exist_ok=True)
for class_name in selected_classes:
    class_path = os.path.join(source_dir, class_name)
    target_path = os.path.join(target_dir, class_name)
    os.makedirs(target_path, exist_ok=True)
    for i, file_name in enumerate(os.listdir(class_path)):
        if i >= images_per_class:
            break
        full_file_name = os.path.join(class_path, file_name)
        if os.path.isfile(full_file_name):
            cv2.imwrite(os.path.join(target_path, file_name), cv2.imread(full_file_name))
print("Veri seti hazırlandı ve dengelendi.")


Veri seti hazırlandı ve dengelendi.


## 2.2 Görüntülerin Boyutlandırılması ve Normalizasyonu

In [3]:
# Görüntülerin işlenmesi (128x128 boyutlandırma ve normalizasyon)
def load_and_process_images(data_dir, image_size=(128, 128)):
    images = []
    labels = []
    for class_name in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_name)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                img = cv2.imread(file_path)
                if img is not None:
                    img_resized = cv2.resize(img, image_size)
                    img_normalized = img_resized / 255.0
                    images.append(img_normalized)
                    labels.append(class_name)
    return np.array(images), np.array(labels)

data_dir = target_dir
X, y = load_and_process_images(data_dir)
print(f"Veri kümesi boyutu: {X.shape}")


Veri kümesi boyutu: (6500, 128, 128, 3)


## 2.3 Verilerin Eğitim ve Test Seti Olarak Ayrılması

In [4]:
# Etiketlerin dönüştürülmesi
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Veriyi bölme (%70 eğitim, %30 test)
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.3, random_state=42)
print(f"Eğitim veri boyutu: {X_train.shape}, Test veri boyutu: {X_test.shape}")


Eğitim veri boyutu: (4550, 128, 128, 3), Test veri boyutu: (1950, 128, 128, 3)


# 3. Veri Artırımı (Augmentation)


In [5]:
# Veri artırma işlemi
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)


# 4. CNN Modelinin Oluşturulması

In [6]:
# CNN Modeli
from tensorflow.keras.layers import Input

model = Sequential([
    Input(shape=(128, 128, 3)),  # Burada Input kullanılıyor
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])


# 5. Modelin Eğitilmesi

In [7]:
# Modeli derleme
model.compile(
    optimizer='adam',               # Optimizasyon algoritması
    loss='categorical_crossentropy', # Kayıp fonksiyonu
    metrics=['accuracy']            # Değerlendirme metriği
)

# Veri artırma (Data Augmentation)
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Eğitim ve doğrulama jeneratörleri
train_generator = datagen.flow(X_train, y_train, batch_size=32)
validation_generator = datagen.flow(X_test, y_test, batch_size=32)

# steps_per_epoch ve validation_steps hesaplama
steps_per_epoch = len(X_train) // 32
validation_steps = len(X_test) // 32

# Modeli eğitme
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)


Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


142/142 ━━━━━━━━━━━━━━━━━━━━ 70s 475ms/step - accuracy: 0.2074 - loss: 4.8779 - val_accuracy: 0.3781 - val_loss: 1.7264
Epoch 2/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - accuracy: 0.4375 - loss: 1.5286 - val_accuracy: 0.3000 - val_loss: 1.8325
Epoch 3/20


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


142/142 ━━━━━━━━━━━━━━━━━━━━ 70s 487ms/step - accuracy: 0.4323 - loss: 1.6808 - val_accuracy: 0.4609 - val_loss: 1.5937
Epoch 4/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step - accuracy: 0.4688 - loss: 1.4582 - val_accuracy: 0.5000 - val_loss: 1.5037
Epoch 5/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 69s 480ms/step - accuracy: 0.4772 - loss: 1.5362 - val_accuracy: 0.4625 - val_loss: 1.5686
Epoch 6/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - accuracy: 0.4688 - loss: 1.6032 - val_accuracy: 0.4667 - val_loss: 1.5562
Epoch 7/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 69s 476ms/step - accuracy: 0.4927 - loss: 1.4787 - val_accuracy: 0.4938 - val_loss: 1.4662
Epoch 8/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - accuracy: 0.5938 - loss: 1.3636 - val_accuracy: 0.3667 - val_loss: 1.6959
Epoch 9/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 81s 476ms/step - accuracy: 0.5297 - loss: 1.3786 - val_accuracy: 0.5000 - val_loss: 1.4489
Epoch 10/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step - accuracy: 0.5625 - loss: 1.5082 - val_ac

# 6. Modelin Test Edilmesi

In [8]:
# Test doğruluğunu değerlendirme
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Doğruluğu: {test_accuracy * 100:.2f}%")


61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.5700 - loss: 1.2738
Test Doğruluğu: 57.79%


# 7. Manipüle Edilmiş Test Seti

In [9]:
# Görüntü manipülasyonu (örneğin parlaklık artırma)
def manipulate_images(images):
    manipulated_images = []
    for img in images:
        manipulated = cv2.convertScaleAbs(img, alpha=1.5, beta=30)  # Parlaklık artırma
        manipulated_images.append(manipulated)
    return np.array(manipulated_images)

X_test_manipulated = manipulate_images(X_test)
manipulated_loss, manipulated_accuracy = model.evaluate(X_test_manipulated, y_test)
print(f"Manipüle Edilmiş Test Doğruluğu: {manipulated_accuracy * 100:.2f}%")


61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.0969 - loss: 223.8763
Manipüle Edilmiş Test Doğruluğu: 9.69%


# 8. Renk Sabitliği (Gray World Algoritması)


In [10]:
def apply_gray_world(image):
    avg_b, avg_g, avg_r = cv2.mean(image)[:3]
    gray_value = (avg_b + avg_g + avg_r) / 3
    scaling_factors = np.array([gray_value / avg_b, gray_value / avg_g, gray_value / avg_r])
    corrected_image = image * scaling_factors  # Her kanal için ayrı ölçekleme
    corrected_image = np.clip(corrected_image, 0, 255).astype(np.uint8)
    return corrected_image
# Manipüle edilmiş test setine renk sabitliği uygulama
X_test_corrected = np.array([apply_gray_world(img) for img in X_test_manipulated])

# Modeli renk sabitliği uygulanmış test setiyle değerlendirme
corrected_loss, corrected_accuracy = model.evaluate(X_test_corrected, y_test)
print(f"Renk Sabitliği Uygulanmış Test Doğruluğu: {corrected_accuracy * 100:.2f}%")


61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.0969 - loss: 225.1878
Renk Sabitliği Uygulanmış Test Doğruluğu: 9.69%


# 9. Sonuçların Karşılaştırılması

In [11]:
# Test sonuçlarının karşılaştırılması
print("Sonuçlar:")
print(f"Orijinal Test Seti Doğruluğu: {test_accuracy * 100:.2f}%")
print(f"Manipüle Edilmiş Test Seti Doğruluğu: {manipulated_accuracy * 100:.2f}%")
print(f"Renk Sabitliği Uygulanmış Test Seti Doğruluğu: {corrected_accuracy * 100:.2f}%")


Sonuçlar:
Orijinal Test Seti Doğruluğu: 57.79%
Manipüle Edilmiş Test Seti Doğruluğu: 9.69%
Renk Sabitliği Uygulanmış Test Seti Doğruluğu: 9.69%
